<a href="https://colab.research.google.com/github/UlkuTuncerKucuktas/mmclassification-MURA/blob/main/MMclassification_MURA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycuda
import torch
import pycuda.driver as cuda
cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name() # '0' is the id of your GPU

     |████████████████████████████████| 1.6MB 8.7MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620900 sha256=1a2b9ac20a84df65e5871f8781979120a4b70a4fcfebbefed9834986759e9b5b
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=91061b1271613f3ed56b643bb40c4d063c7cc337f5ea9ec8b939da5ff534e298
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


'Tesla P100-PCIE-16GB'

In [2]:
!pip install -q mmcv terminaltables
import mmcv

     |████████████████████████████████| 256kB 8.5MB/s 
     |████████████████████████████████| 194kB 13.5MB/s 


In [3]:
!pip install mmcv-full==latest+torch1.7.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
     |████████████████████████████████| 19.8MB 159kB/s 


In [4]:
!git clone https://github.com/open-mmlab/mmclassification.git

Cloning into 'mmclassification'...
remote: Enumerating objects: 1413, done.
remote: Total 1413 (delta 0), reused 0 (delta 0), pack-reused 1413
Receiving objects: 100% (1413/1413), 581.97 KiB | 15.73 MiB/s, done.
Resolving deltas: 100% (835/835), done.


In [5]:
%cd mmclassification
!pip install -e .


/content/mmclassification
Obtaining file:///content/mmclassification
  Running setup.py develop for mmcls


In [6]:


f= open("/content/mmclassification/configs/myconfig.py","w+")


configfile = """


_base_ = [
    '/content/mmclassification/configs/_base_/models/resnest50.py',
    '/content/mmclassification/configs/_base_/datasets/mydataset.py', '/content/mmclassification/configs/_base_/default_runtime.py'
]

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

# optimizer
# lr is set for a batch size of 128
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)
# learning policy
lr_config = dict(
    policy='step',
    step=[15])
total_epochs = 20
log_config = dict(interval=100)

dataset_A_train = dict(
    type='Filelist',
    ann_file = '/content/mmclassification/trainann.txt',
    pipeline=train_pipeline
)


"""


f.write(configfile)
f.close()





In [7]:
!python /content/mmclassification/configs/myconfig.py

In [8]:
!unzip /content/drive/MyDrive/Mura_Dataset.rar 

Görüntülenen çıkış son 5000 satıra kısaltıldı.
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positive/image1.

In [9]:
import pandas as pd

TrainCsvFile = pd.read_csv("/content/mmclassification/MURA-v1.1/train_image_paths.csv", names = ["Path"])
ValidCsvFile = pd.read_csv("/content/mmclassification/MURA-v1.1/valid_image_paths.csv", names = ["Path"])


In [10]:
import PIL
from PIL import Image
from matplotlib.pyplot import imshow
import numpy as np
from tqdm import tqdm


positivetrainpath = "/content/mmclassification/data/Muradataset/train/positive"

negativetrainpath = "/content/mmclassification/data/Muradataset/train/negative"

positivevalidpath = "/content/mmclassification/data/Muradataset/val/"

negativevalidpath = "/content/mmclassification/data/Muradataset/val/"

dirs = [positivetrainpath,negativetrainpath,positivevalidpath,negativevalidpath]
import os

f = open("trainann.txt","w+")
q = open("valann.txt","w+")


for dir in dirs:
  try:
    os.makedirs(dir)
  except:
    pass
#Pytorch kütüphenesinde bulunan ImageFolder dataframe hazırlama fonksiyonundan yararlanmak için
#bizim elimizde bulunan data directory dizini üzerinde değişiklikler yapmak amacı ile yardımcı
#fonksiyon

def ImageFolderCreator(csvfilepath ,positivepath,negativepath,ann):
  for i in tqdm(range(csvfilepath.shape[0])):
    img = Image.open("/content/mmclassification/" + csvfilepath[i])
    img = img.resize((224,224))
    img = img.convert('RGB')
    
    if "positive" in csvfilepath[i]:
      img.save(positivepath + "/img{}.png".format(i),"PNG")
      ann.write("img{}.png".format(i) + " " + "1" + "\n")
    if "negative" in csvfilepath[i]:
      img.save(negativepath + "/img{}.png".format(i),"PNG")
      ann.write("img{}.png".format(i) + " " + "0" + "\n")

ImageFolderCreator(TrainCsvFile['Path'],positivetrainpath,negativetrainpath,f)
ImageFolderCreator(ValidCsvFile['Path'],positivevalidpath,negativevalidpath,q)

f.close()
q.close()
  

100%|██████████| 3197/3197 [00:51<00:00, 61.93it/s]


In [11]:
datasetstring = """


# dataset settings
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=224, backend='pillow'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, -1), backend='pillow'),
    dict(type='CenterCrop', crop_size=224),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]
data = dict(
    samples_per_gpu=20,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        data_prefix='/content/mmclassification/data/Muradataset/train',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_prefix='/content/mmclassification/data/Muradataset/val',
        ann_file='/content/mmclassification/valann.txt',
        pipeline=test_pipeline),
    test=dict(
        # replace `data/val` with `data/test` for standard test
        type=dataset_type,
        data_prefix='/content/mmclassification/data/Muradataset/val',
        ann_file='/content/mmclassification/valann.txt',
        pipeline=test_pipeline))
evaluation = dict(interval=1, metric='accuracy')




"""


with open("/content/mmclassification/configs/_base_/datasets/mydataset.py","w+") as f:
  f.write(datasetstring)
  f.close()
  

In [16]:
accuracy = """

import numpy as np
import torch
import torch.nn as nn


def accuracy_numpy(pred, target, topk):
    res = []
    maxk = max(topk)
    num = pred.shape[0]
    pred_label = pred.argsort(axis=1)[:, -maxk:][:, ::-1]

    for k in topk:
        correct_k = np.logical_or.reduce(
            pred_label[:, :k] == target.reshape(-1,1), axis=1)
        res.append(correct_k.sum() * 100. / num)
    return res


def accuracy_torch(pred, target, topk=1):
    res = []
    maxk = max(topk)
    num = pred.size(0)
    _, pred_label = pred.topk(maxk, dim=1)
    pred_label = pred_label.t()
    correct = pred_label.eq(target.view(1, -1).expand_as(pred_label))

    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100. / num))
    return res


def accuracy(pred, target, topk=1):
    assert isinstance(topk, (int, tuple))
    if isinstance(topk, int):
        topk = (topk, )
        return_single = True
    else:
        return_single = False

    if isinstance(pred, torch.Tensor) and isinstance(target, torch.Tensor):
        res = accuracy_torch(pred, target, topk)
    elif isinstance(pred, np.ndarray) and isinstance(target, np.ndarray):
        res = accuracy_numpy(pred, target, topk)
    else:
        raise TypeError('pred and target should both be'
                        'torch.Tensor or np.ndarray')

    return res[0] if return_single else res


class Accuracy(nn.Module):

    def __init__(self, topk=(1, )):
        super().__init__()
        self.topk = topk

    def forward(self, pred, target):
        return accuracy(pred, target, self.topk)




"""

with open("/content/mmclassification/mmcls/models/losses/accuracy.py","w+") as f:
  f.write(accuracy)
  f.close()


In [17]:
# Accuracy dosyasında .view(-1) i .reshape(-1) yap
!python "/content/mmclassification/tools/train.py" "/content/mmclassification/configs/myconfig.py"

2020-11-29 18:18:17,889 - mmcls - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.6.9 (default, Oct  8 2020, 12:12:24) [GCC 8.4.0]
CUDA available: True
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.1, V10.1.243
GPU 0: Tesla P100-PCIE-16GB
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.0+cu101
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75
  - C